In [1]:
import tensorflow as tf 
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle

In [3]:
# Authenticate to your cloud account
from google.colab import auth
auth.authenticate_user()

In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/topic classification data set/train.csv',names=['classes_index','title','description'])
test_data = pd.read_csv('/content/drive/MyDrive/topic classification data set/test.csv',names=['classes_index','title','description'])

In [5]:
test_data = shuffle(test_data, random_state=22)
train_data = shuffle(train_data, random_state=22)

In [6]:
len(train_data)

120000

In [7]:
train_data.replace(to_replace = 1,  
                 value = "World",  
                  inplace = True)
train_data.replace(to_replace = 2,  
                 value = "Sports",  
                  inplace = True)
train_data.replace(to_replace = 3,  
                 value = "Business",  
                  inplace = True)
train_data.replace(to_replace = 4,  
                 value = "Sci/Tech",  
                  inplace = True)

In [8]:
test_data.replace(to_replace = 1,  
                 value = "World",  
                  inplace = True)
test_data.replace(to_replace = 2,  
                 value = "Sports",  
                  inplace = True)
test_data.replace(to_replace = 3,  
                 value = "Business",  
                  inplace = True)
test_data.replace(to_replace = 4,  
                 value = "Sci/Tech",  
                  inplace = True)

In [9]:
train_data = train_data.drop(columns=['title'])
test_data = test_data.drop(columns=['title'])

In [10]:
labels = train_data["classes_index"].to_numpy() # convert labels column to NumPy array
labels[:10]

array(['Sci/Tech', 'Business', 'Sci/Tech', 'Business', 'Business',
       'World', 'Sci/Tech', 'Sports', 'World', 'World'], dtype=object)

In [11]:
unique_breeds = np.unique(labels)
len(unique_breeds)

4

In [12]:
boolean_labels = [label == np.array(unique_breeds) for label in labels]
boolean_labels[:3]

[array([False,  True, False, False]),
 array([ True, False, False, False]),
 array([False,  True, False, False])]

In [13]:
print(labels[0]) # original label
print(np.where(unique_breeds == labels[0])[0][0]) # index where label occurs
print(boolean_labels[0].argmax()) # index where label occurs in boolean array
print(boolean_labels[0].astype(int)) # there will be a 1 where the sample label occurs

Sci/Tech
1
1
[0 1 0 0]


In [14]:
transactions_train = [news for news in train_data["description"]]
print(transactions_train[:2])

['A broadband wireless network is helping researchers at an important panda park in China.', 'Microsoft will pay Novell \\$536 million to settle an antitrust dispute. Novell plans to file a new suit againt Microsoft regarding word-processing software.']


In [15]:
transactions_test = [news for news in test_data["description"]]
print(transactions_test[:2])

[" PHILADELPHIA (Reuters) - Donald Trump's casino operations  filed for bankruptcy Sunday in a long-expected move that would  allow the real estate maverick to restructure the company's  debt and overhaul its aging casinos.", 'If you read the recent excerpt of "Namath" in Sports Illustrated and were put off by the apparent focus on the iconic Broadway Joe\'s personal life, be comforted in the knowledge that Mark Kriegel\'s 441-page biography includes plenty of football, too. The book is exhaustively researched and includes telling anecdotes from Beaver Falls, Pa., to Tuscaloosa, Ala., to New York.']


In [16]:
labels_test = test_data["classes_index"].to_numpy() # convert labels column to NumPy array
labels_test[:10]

array(['Business', 'Sports', 'Business', 'Sci/Tech', 'Sci/Tech',
       'Sci/Tech', 'Business', 'World', 'Sci/Tech', 'Business'],
      dtype=object)

In [17]:
boolean_labels_test = [label == np.array(unique_breeds) for label in labels_test]
boolean_labels_test[:3]

[array([ True, False, False, False]),
 array([False, False,  True, False]),
 array([ True, False, False, False])]

In [18]:
len(transactions_test), len(boolean_labels_test)

(7600, 7600)

In [19]:
X_test = transactions_test
y_test = boolean_labels_test

In [20]:
X = transactions_train
y = boolean_labels

In [ ]:
num_news =10000
num_news

10000

In [ ]:


# Split them into training and validation using NUM_IMAGES 
X_train, X_val, y_train, y_val = train_test_split(X[:num_news],
                                                  y[:num_news], 
                                                  test_size=0.2,
                                                  random_state=42)


In [ ]:
len(X_train), len(y_train), len(X_val), len(y_val)

(8000, 8000, 2000, 2000)

In [22]:
!pip3 install tensorflow_text

     |████████████████████████████████| 3.4MB 7.4MB/s 


In [23]:
BATCH_SIZE = 32
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image (x) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle it if it's validation data.
  Also accepts test data as input (no labels).
  """
  # If the data is a test dataset, we probably don't have labels
  if test_data:
    print("Creating test data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only filepaths
    data_batch = data.batch(BATCH_SIZE)
    return data_batch
  
  # If the data if a valid dataset, we don't need to shuffle it
  elif valid_data:
    print("Creating validation data batches...")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                               tf.constant(y))) # labels
    data_batch = data.batch(BATCH_SIZE)
    return data_batch

  else:
    # If the data is a training dataset, we shuffle it
    print("Creating training data batches...")
    # Turn filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                              tf.constant(y))) # labels
    
    # Turn the data into batches
    data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [37]:
train_data = create_data_batches(X,y)

Creating training data batches...


In [ ]:
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

Creating training data batches...
Creating validation data batches...


In [ ]:
test_data = create_data_batches(X_test, y_test, valid_data=True)

Creating validation data batches...


In [ ]:
train_text, train_labels = next(iter(train_data))

In [ ]:
train_text[0]
train_labels[0]

<tf.Tensor: shape=(4,), dtype=bool, numpy=array([False, False,  True, False])>

In [ ]:
import collections
import pathlib
import re
import string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [ ]:
int_vectorize_layer.adapt(train_text)

In [ ]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [ ]:
int_train_ds = train_data.map(int_vectorize_text)
int_val_ds = val_data.map(int_vectorize_text)

In [ ]:
int_train_ds.element_spec

(TensorSpec(shape=(None, 250), dtype=tf.int64, name=None),
 TensorSpec(shape=(None, 4), dtype=tf.bool, name=None))

In [ ]:
int_train_text, int_train_labels = next(iter(int_train_ds))

In [ ]:
def create_model(vocab_size, num_tags):
  
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(250, input_shape=(VOCAB_SIZE,MAX_SEQUENCE_LENGTH), activation='relu'))
  model.add(tf.keras.layers.Dense(25, activation='relu'))
  model.add(tf.keras.layers.Dense(num_tags, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
model = create_model(VOCAB_SIZE, 4)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10000, 250)        62750     
_________________________________________________________________
dense_1 (Dense)              (None, 10000, 25)         6275      
_________________________________________________________________
dense_2 (Dense)              (None, 10000, 4)          104       
Total params: 69,129
Trainable params: 69,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(int_train_ds, epochs=100, batch_size=32,validation_data=int_val_ds,validation_freq=1)

Epoch 1/100
25/25 [==============================] - 2s 21ms/step - loss: 8.9476 - accuracy: 0.2376 - val_loss: 2.4483 - val_accuracy: 0.2650
Epoch 2/100
25/25 [==============================] - 0s 6ms/step - loss: 1.9477 - accuracy: 0.3145 - val_loss: 1.0609 - val_accuracy: 0.2900
Epoch 3/100
25/25 [==============================] - 0s 6ms/step - loss: 0.8552 - accuracy: 0.3319 - val_loss: 0.8587 - val_accuracy: 0.2600
Epoch 4/100
25/25 [==============================] - 0s 6ms/step - loss: 0.6584 - accuracy: 0.3380 - val_loss: 0.8446 - val_accuracy: 0.2500
Epoch 5/100
25/25 [==============================] - 0s 6ms/step - loss: 0.6190 - accuracy: 0.3502 - val_loss: 0.8451 - val_accuracy: 0.2500
Epoch 6/100
25/25 [==============================] - 0s 6ms/step - loss: 0.6058 - accuracy: 0.3584 - val_loss: 0.8377 - val_accuracy: 0.2500
Epoch 7/100
25/25 [==============================] - 0s 6ms/step - loss: 0.5928 - accuracy: 0.3633 - val_loss: 0.8293 - val_accuracy: 0.2450
Epoch 8/100


In [25]:
!pip install -q tf-models-official

     |████████████████████████████████| 1.1MB 8.3MB/s 
     |████████████████████████████████| 174kB 48.9MB/s 
     |████████████████████████████████| 706kB 35.7MB/s 
     |████████████████████████████████| 358kB 58.6MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 37.6MB 80kB/s 
     |████████████████████████████████| 102kB 16.3MB/s 
     |████████████████████████████████| 1.2MB 52.7MB/s 
     |████████████████████████████████| 645kB 45.4MB/s 


In [27]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

import matplotlib.pyplot as plt
from google.colab import files

tf.get_logger().setLevel('ERROR')

In [29]:
bert_preprocess_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [30]:
text_test = ['OXFORD, Miss. - A press conference to announce the hiring of a new head football coach at The University of Mississippi has been scheduled for Thursday morning, December 16, at 10:00 am, in the new Indoor Practice Facility.']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_mask', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [  101  4345  1010  3335  1012  1011  1037  2811  3034  2000 14970  1996]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [31]:
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3")

In [32]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {"https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[-0.82245016 -0.5516178  -0.97415954  0.7065633   0.7690096  -0.09132629
  0.5081806   0.46767193 -0.866974   -0.9999939  -0.71754676  0.965547  ]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[-0.5614217  -0.24129212 -0.03888294 ...  0.11010697  0.59314036
   0.22141232]
 [-1.1494048   0.72628284 -0.52658045 ...  0.19967534  0.35520113
  -0.40773112]
 [-1.076503    0.826983   -0.23702139 ... -0.700099    0.495587
   0.09486427]
 ...
 [ 0.45240694  0.11662792  0.81127644 ...  0.19132312  0.32670778
   0.25141114]
 [-0.4230254   0.2862644   1.406711   ... -0.24142835 -0.22575398
   0.8432041 ]
 [-0.35000107 -0.32413366  0.32496807 ...  0.3665411  -0.24517196
   0.23549658]]


In [33]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(4, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [34]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.37908483 0.37341616 0.22873497 0.48573288]], shape=(1, 4), dtype=float32)


In [38]:
epochs = 20
steps_per_epoch = tf.data.experimental.cardinality(train_data).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [40]:
classifier_model.compile(optimizer=optimizer,
                         loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                         metrics=["accuracy"])

In [41]:
%load_ext tensorboard
import datetime

# Create a function to build a TensorBoard callback
def create_tensorboard_callback():
  # Create a log directory for storing TensorBoard logs
  logdir = os.path.join("drive/My Drive/Data/logs",
                        # Make it so the logs get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)
tensorboard = create_tensorboard_callback()

In [ ]:
cfcchgchgchfhgfgfhgfgf

In [39]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="accuracy",
                                                  patience=3)

In [ ]:
print(f'Training model with {"https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"}')
history = classifier_model.fit(x=train_data,
                               epochs=epochs,
                               callbacks = [tensorboard,early_stopping])

Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/20
3750/3750 [==============================] - 3723s 989ms/step - loss: 0.6406 - accuracy: 0.7316
Epoch 2/20
3750/3750 [==============================] - 3701s 987ms/step - loss: 0.2011 - accuracy: 0.9317
Epoch 3/20
3750/3750 [==============================] - 3703s 987ms/step - loss: 0.1497 - accuracy: 0.9531
Epoch 4/20
3750/3750 [==============================] - 3687s 983ms/step - loss: 0.1041 - accuracy: 0.9693
Epoch 5/20
3750/3750 [==============================] - 3681s 982ms/step - loss: 0.0734 - accuracy: 0.9794
Epoch 6/20
3750/3750 [==============================] - 3679s 981ms/step - loss: 0.0582 - accuracy: 0.9846
Epoch 7/20
3750/3750 [==============================] - 3678s 981ms/step - loss: 0.0446 - accuracy: 0.9887
Epoch 8/20
 418/3750 [==>...........................] - ETA: 54:31 - loss: 0.0382 - accuracy: 0.9898

In [ ]:
test_data.element_spec

(TensorSpec(shape=(None,), dtype=tf.string, name=None),
 TensorSpec(shape=(None, 4), dtype=tf.bool, name=None))

In [2]:
loss, accuracy = classifier_model.evaluate(test_data)

NameError: ignored

In [3]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

fig.show()

fig.savefig('samplefigure1.png')
files.download('samplefigure1.png')

NameError: ignored

In [ ]:
predictions = classifier_model.predict(val_data, verbose=1) # verbose shows us how long there is to go
predictions

63/63 [==============================] - 20s 315ms/step


array([[-4.6911387 ,  8.078234  , -5.681418  ,  2.7777278 ],
       [ 0.21572642, 12.348355  , -6.17419   , -5.0808654 ],
       [-0.31314227, 12.3722315 , -4.5130486 , -5.733709  ],
       ...,
       [-4.858727  , -3.0327249 , 11.914003  , -2.3165758 ],
       [-5.0573287 , -3.9286394 , -4.239731  , 10.802811  ],
       [11.108394  , -2.7709186 , -4.269663  , -3.8393319 ]],
      dtype=float32)

In [ ]:
print(predictions[0])
print(f"Max value (probability of prediction): {np.max(predictions[0])}") # the max probability value predicted by the model
print(f"Sum: {np.sum(predictions[0])}") # because we used softmax activation in our model, this will be close to 1
print(f"Max index: {np.argmax(predictions[0])}") # the index of where the max value in predictions[0] occurs
print(f"Predicted label: {unique_breeds[np.argmax(predictions[0])]}") # the predicted label

[-4.6911387  8.078234  -5.681418   2.7777278]
Max value (probability of prediction): 8.07823371887207
Sum: 0.48340487480163574
Max index: 1
Predicted label: Sci/Tech


In [ ]:
val_text , val_labels = next(iter(val_data))
for i in [1,17,21]:
    print("actual sentence")
    print(val_text[i])
    print("actual label")
    print(val_labels[i])
    print("predictions")
    print(predictions[i])
    print(f"Max value (probability of prediction): {np.max(predictions[i])}") # the max probability value predicted by the model
    print(f"Sum: {np.sum(predictions[i])}") # because we used softmax activation in our model, this will be close to 1
    print(f"Max index: {np.argmax(predictions[i])}") # the index of where the max value in predictions[0] occurs
    print(f"Predicted label: {unique_breeds[np.argmax(predictions[i])]}") # the predicted label

actual sentence
tf.Tensor(b'The two tech giants bring corporate-level customer management software to mom-and-pop shops.', shape=(), dtype=string)
actual label
tf.Tensor([False  True False False], shape=(4,), dtype=bool)
predictions
[ 0.21572642 12.348355   -6.17419    -5.0808654 ]
Max value (probability of prediction): 12.348355293273926
Sum: 1.3090267181396484
Max index: 1
Predicted label: Sci/Tech
actual sentence
tf.Tensor(b'The United States #39; DaMarcus Beasley (17), who plays for PSV Einhoven in the Netherlands, battles for the ball against Panama #39;s Jose Anthony Torres, left, during the second half of their World Cup qualifying match Wednesday, Oct.', shape=(), dtype=string)
actual label
tf.Tensor([False False  True False], shape=(4,), dtype=bool)
predictions
[-4.464979  -3.7125914 12.62537   -3.2285938]
Max value (probability of prediction): 12.625370025634766
Sum: 1.2192058563232422
Max index: 2
Predicted label: Sports
actual sentence
tf.Tensor(b'NEW YORK (CNN/Money) - Gen

In [ ]:
unique_breeds

array(['Business', 'Sci/Tech', 'Sports', 'World'], dtype=object)

In [ ]:

def save_model(model, suffix=None):
  """
  Saves a given model in a models directory and appends a suffix (str)
  for clarity and reuse.
  """
  # Create model directory with current time
  modeldir = os.path.join("drive/My Drive/Data/models",
                          datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
  model_path = modeldir + "-" + suffix + ".h5" # save format of model
  print(f"Saving model to: {model_path}...")
  model.save(model_path)
  return model_path

In [ ]:
save_model(classifier_model, suffix="10000")

Saving model to: drive/My Drive/Data/models/20210211-22151613081754-10000.h5...


'drive/My Drive/Data/models/20210211-22151613081754-10000.h5'

In [1]:
def load_model(model_path):
  """
  Loads a saved model from a specified path.
  """
  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path,
                                     custom_objects={"KerasLayer":hub.KerasLayer})
  return model

In [35]:
loaded_full_model = load_model('drive/MyDrive/Data/models/20210211-22151613081754-10000.h5')

Loading saved model from: drive/MyDrive/Data/models/20210211-22151613081754-10000.h5


ValueError: ignored